In [18]:
import numpy as np
import pandas as pd
import pyesg
from arrays import invest_array, salary_array, age_array
from allocation import allocation_array

periods_per_year = 2
years = 40

age = 30
ret_age = 65
fund_value_start = 180000
salary = 190000
salary_growth = .045
invest_pct = .15
inflation = .022
stock_pct_strt = .80
bond_pct_strt = 1 - stock_pct_strt
five_yr_pct = .55
stock_pct_end = .2
bond_pct_end = 1 - stock_pct_end

# instantiate a new model with the required parameters
stock_model = pyesg.GeometricBrownianMotion(mu=0.10, sigma=0.15)
bond_model = pyesg.GeometricBrownianMotion(mu=0.06, sigma=0.05)

x0 = 100.0                          # the start value of our process
dt = 1/periods_per_year            # the length of each timestep in years
n_scenarios = 2                     # the number of scenarios to generate
n_steps = periods_per_year * years # the number of time steps per scenario
random_state = 259                  # optional random_state for reproducibility

s_model_results = stock_model.scenarios(x0, dt, n_scenarios, n_steps, random_state)
b_model_results = bond_model.scenarios(x0, dt, n_scenarios, n_steps, random_state)

stock_return = s_model_results[:, 1:] / s_model_results[:, :-1]
bond_return = b_model_results[:, 1:] / b_model_results[:, :-1]

stock_array = np.insert(stock_return, 0, fund_value_start * stock_pct_strt, axis=1)
bond_array = np.insert(bond_return, 0, fund_value_start * bond_pct_strt, axis=1)

salary_arr = salary_array(salary = salary, salary_growth= salary_growth, periods_per_year=periods_per_year, years=years)
invest_arr = invest_array(salary_array = salary_arr, invest_pct = invest_pct, periods_per_year = periods_per_year)
allocation_arr = allocation_array(
    start_pct = stock_pct_strt,        
    five_yr_pct = five_yr_pct,
    end_pct = stock_pct_end,
    age = age,             
    ret_age = ret_age,    
    periods_per_year = periods_per_year,   
    years = years 
)            


for s, b, inv, alloc_s, alloc_b in zip(stock_array, bond_array, invest_arr, allocation_arr[1], allocation_arr[2]):
    for i in range(1, len(s)):
        s[i] = s[i-1] * s[i]
        b[i] = b[i-1] * b[i]
        total_fund = s[i] + b[i] + invest_arr[i-1]/periods_per_year
        s[i] = total_fund * alloc_s
        b[i] = total_fund * alloc_b

age_arr = age_array(age = age, periods_per_year=periods_per_year, years=years)

age_names = ['age_' + str(i+1) for i in range(len(age_array().reshape(1, -1)))]
stock_fund_names = ['stock_fund_value_' + str(i+1) for i in range(len(stock_array))]
stock_return_names = ['stock_return_' + str(i+1) for i in range(len(stock_array))]
bond_fund_names = ['bond_fund_value_' + str(i+1) for i in range(len(bond_array))]
bond_return_names = ['bond_return_' + str(i+1) for i in range(len(bond_array))]

age_array_df = pd.DataFrame(age_array().reshape(1, -1), index=age_names)
stock_array_df = pd.DataFrame(stock_array, index=stock_fund_names)
stock_return_df = pd.DataFrame(stock_return, index=stock_return_names)
bond_array_df = pd.DataFrame(bond_array, index=bond_fund_names)
bond_return_df = pd.DataFrame(bond_return, index=bond_return_names)
# salary_array_df = pd.DataFrame(salary_array)
# invest_array_df = pd.DataFrame(invest_array)

df = pd.concat([age_array_df, stock_array_df, stock_return_df, bond_array_df, bond_return_df], axis=0)
with pd.ExcelWriter('Outputs/retirement_calc.xlsx') as writer:
    df.to_excel(writer)
    # stock_array_df.to_excel(writer, sheet_name='stock_array')
    # bond_array_df.to_excel(writer, sheet_name='bond_array')
    # stock_return_df.to_excel(writer, sheet_name='stock_return')
    # bond_return_df.to_excel(writer, sheet_name='bond_return')
    # index_return_df.to_excel(writer, sheet_name='index')
    # salary_array_df.to_excel(writer, sheet_name='salary')
    # invest_array_df.to_excel(writer, sheet_name='invest')
    # age_array_df.to_excel(writer, sheet_name='age')

In [50]:
invest_pct_per_period = invest_pct / periods_per_year
salary_growth_per_period = (1+salary_growth)**(1/periods_per_year)-1
invest_pct_per_period = (1+invest_pct)**(1/periods_per_year)-1

stock_alloc_array = np.ones(periods_per_year * years + 1) * (stock_pct_e - stock_pct_strt) / (ret_age - age) / periods_per_year
stock_alloc_array[0] = stock_pct_strt
stock_alloc_array = np.cumsum(stock_alloc_array)

age_array = np.ones(periods_per_year * years + 1) / periods_per_year
age_array[0] = age
age_array = np.cumsum(age_array) 

salary_growth_array = np.ones(periods_per_year * years) * (1+salary_growth_per_period)
salary_growth_array[0] = salary
salary_array = np.cumprod(salary_growth_array)
invest_array = salary_array * invest_pct


0
1
2
